In [62]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
from helpers import load_csv_data

# load data.
PATH = "data/dataset/"
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data/dataset/", sub_sample=False)

In [49]:
#weight_ratio, filling nan with mean
weight_ratio = x_train[:,252]
weight_ratio_filled = weight_ratio
weight_ratio_filled[np.isnan(weight_ratio_filled)]=np.mean(weight_ratio_filled[~np.isnan(weight_ratio_filled)])
weight_ratio_filled

array([20.78      , 28.7       , 28.03300679, ..., 58.53      ,
       35.87      , 22.81      ])

In [50]:
#age5y, filling nan with mean
age = x_train[:,245]
age_filled = age
age_filled[np.isnan(age_filled)]=np.mean(age_filled[~np.isnan(age_filled)])
age_filled

array([ 8.,  8., 10., ..., 10.,  5.,  7.])

In [51]:
x_train = np.c_[weight_ratio_filled, age_filled]
x_train.shape

(328135, 2)

In [52]:
#Selecting an equal number of people with heart attack and those who has not

# Identifiez les indices des patients avec un arrêt cardiaque (y=1) et sans (y=0)
indices_y1 = np.where(y_train == 1)[0]
indices_y0 = np.where(y_train == 0)[0]



In [53]:

# Sélectionnez un nombre égal d'indices aléatoires pour les deux classes
num_samples = min(len(indices_y1), len(indices_y0))
selected_indices_y1 = np.random.choice(indices_y1, num_samples, replace=False)
selected_indices_y0 = np.random.choice(indices_y0, num_samples, replace=False)

print('indices selectionés:', selected_indices_y1)
print('indices selectionés y0:', selected_indices_y0)

# Combinez les indices sélectionnés pour former un ensemble de données équilibré
selected_indices = np.concatenate((selected_indices_y1, selected_indices_y0))

# Sélectionnez les données correspondantes à ces indices 
balanced_x_train = x_train[selected_indices]
balanced_y_train = y_train[selected_indices]

indices selectionés: [ 55063 138932 200666 ...  99397 174821  67664]
indices selectionés y0: [ 90032 136594  58090 ... 216880 234709  46085]


In [54]:
balanced_x_train.shape

(57950, 2)

In [90]:
from helpers import standardize
balanced_x_train_std = np.c_[standardize(balanced_x_train[:,0]).reshape(-1), standardize(balanced_x_train[:,1]).reshape(-1)]
balanced_x_train_std


array([[ 1.23872596, -0.88375283],
       [-0.29796388,  1.24659145],
       [ 0.02781437,  0.02925186],
       ...,
       [-0.0828273 ,  0.02925186],
       [-0.89880961, -2.40542732],
       [-0.36404154, -0.57941793]])

In [91]:

X = balanced_x_train_std
y = balanced_y_train
# Specify the proportion of data to use for the test set (in this example, 20%)
test_size = 0.2
split_index = int(len(X) * (1 - test_size))

# Split the data into training and testing sets
X_train, X_test_in = X[:split_index], X[split_index:]
y_train, y_test_in = y[:split_index], y[split_index:]

# You can now use X_train and y_train to train your model
# and X_test and y_test to evaluate the model's performance.

(46360, 7)

In [104]:
from implementations import ridge_regression

In [105]:
from helpers import build_poly

In [109]:
from cross_validation import *

best_lambda, best_rmse = cross_validation_demo(3, 3, np.logspace(-4, 0, 30), y_train, X_train)



For polynomial expansion up to degree 3, the choice of lambda which leads to the best test rmse is 0.00010 with a test rmse of 0.435


In [110]:
#test 

X_poly = build_poly(X_train, 3)
w_opt = ridge_regression(y_train, X_poly, best_lambda)

In [111]:
X_test_poly = build_poly(X_test_in, 3)
y_pred = np.dot(X_test_poly, w_opt)

In [112]:
from implementations import compute_mse

mse = compute_mse(y_train, X_poly, w_opt)
test_mse = compute_mse(y_test_in, X_test_poly, w_opt)
mse, test_mse

(0.09458636971759167, 0.1565122840591741)

In [113]:
y_pred[y_pred<0.5]=0
y_pred[y_pred>=0.5]=1
len(y_pred), sum(y_pred)

(11590, 6495.0)